In [1]:
# Imports
import os
import json
import requests
import gradio as gr
from typing import cast
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader

# OpenAI types
from openai.types.chat import ChatCompletionMessageParam

# Markdown output display
from IPython.display import Markdown, display

In [2]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [3]:
# Load Environment Variables
load_dotenv(override=True)

# Gemini Environment Variables
gemini_api_key = env_to_str('GEMINI_API_KEY')
gemini_base_url = env_to_str('GEMINI_BASE_URL')
model_gemini_flash = env_to_str('MODEL_GEMINI_FLASH')

# Gemini Client
gemini = OpenAI(
    api_key=gemini_api_key,
    base_url=gemini_base_url
)

In [4]:
# List for storing notifications instead of using Pushover
notifications = []

In [6]:
# Push Function
def push(message):
    """
    Prints a message and adds it to the global notifications list.
    """
    print(f"Notification: {message}")
    notifications.append(message)

# Example of the new push function
push("New Notification!")
print(notifications)

Notification: New Notification!
['New Notification!']
